In [1]:
import requests
from bs4 import BeautifulSoup as bs
import re
import pandas as pd
import numpy as np
import json

In [2]:
class Links:
    def __init__(self, link):
#       Web scrapping initial setups
        self.link = requests.get(link)
        self.soup = bs(self.link.content)
        
    def get_main_links(self):
        ul = self.soup.find_all('ul')[1]
        a = ul.find_all('a')

        links = list()
        years = ['2000', '2010']

        for item in a:
            for year in years:
                if year in item.get('href'):
                    links.append(item.get('href'))
        
        return links
                    
          
    def get_movie_links(self):
        main_links = self.get_main_links()
        movies_links = list()

        for link in main_links:
            req = requests.get(f"https://en.wikipedia.org/{link}")
            content = bs(req.content)

            table = content.find('tbody')
            i = table.find_all('i')
            
            for item in i:
                a = item.find('a')
                if a != None:
                    movies_links.append(a.get('href'))
                else:
                    continue
            
        return movies_links

In [3]:
links = Links("https://en.wikipedia.org/wiki/Lists_of_Lionsgate_films")
all_links = links.get_movie_links()

### Load data to json file

In [5]:
class Load:
    def __init__(self, urls):
        self.urls = urls
        self.base_path = "https://en.wikipedia.org/"
        
    def get_data(self):
        for url in self.urls:
            req = requests.get(self.base_path + url)
            content = bs(req.content)

            keys = content.select('table.infobox.vevent tbody tr th')
            values = content.select('table.infobox.vevent tbody tr td')
            data = dict()
            for key, value in zip(keys, values):
                print(f'{key.get_text(" ", strip=True)}')
            
#             return values

In [6]:
load = Load(all_links[:2])
load.get_data()

American Psycho
Directed by
Screenplay by
Based on
Produced by
Starring
Cinematography
Edited by
Music by
Production companies
Distributed by
Release date
Running time
Countries
Language
Budget
Box office
The Big Kahuna
Directed by
Screenplay by
Based on
Produced by
Starring
Cinematography
Edited by
Music by
Production companies
Distributed by
Release date
Running time
Country
Language
Budget
Box office
